**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/hardware.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  urlretrieve("https://raw.githubusercontent.com/RobotLocomotion/drake/master/tools/install/colab/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version='20200805', build='nightly') 

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Helper methods that should get PR'd to drake.

import asyncio
import sys
from IPython import get_ipython
from warnings import warn


# Note: The implementation below was inspired by
# https://github.com/Kirill888/jupyter-ui-poll , though I suspect it can be
# optimized.
#
# For reference,
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html  # noqa
# describes the problem but it only offers the solution of using separate 
# threads for execution; a workflow that we do not wish to impose on users.

def process_ipywidget_events(num_events_to_process=1):
    """
    Allows the kernel to process GUI events.  This is required in order to
    process ipywidget updates inside a simulation loop.
    """

    shell = get_ipython()
    # Ok to do nothing if running from console
    if shell is None:
        return
    kernel = shell.kernel
    events = []
    kernel.shell_handlers['execute_request'] = lambda *e: events.append(e)
    current_parent = (kernel._parent_ident, kernel._parent_header)

    for _ in range(num_events_to_process):
        # ensure stdout still happens in the same cell
        kernel.set_parent(*current_parent)
        kernel.do_one_iteration()
        kernel.set_parent(*current_parent)

    kernel.shell_handlers['execute_request'] = kernel.execute_request

    def _replay_events(shell, events):
        kernel = shell.kernel
        sys.stdout.flush()
        sys.stderr.flush()
        for stream, ident, parent in events:
            kernel.set_parent(ident, parent)
            if kernel._aborting:
                kernel._send_abort_reply(stream, parent, ident)
            else:
                kernel.execute_request(stream, ident, parent)

    if len(events) > 0:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.call_soon(lambda: _replay_events(shell, events))
        else:
            warn('Automatic execution of scheduled cells only works with '
                'asyncio-based ipython')


import numpy as np
from IPython.display import display
from ipywidgets import FloatSlider, Layout
from pydrake.multibody.tree import JointIndex
from pydrake.systems.framework import VectorSystem

class JointSliders(VectorSystem):
    """
    Provides a simple ipywidgets gui with one slider per joint of the
    MultibodyPlant.  Any positions that are not associated with joints (e.g.
    floating-base "mobilizers") are held constant at the default value
    obtained from robot.CreateDefaultContext().

    System YAML
        name: JointSliders
        output_ports:
        - positions
    """

    def __init__(self, robot, lower_limit=-10., upper_limit=10.,
                 resolution=0.01, length=200, update_period_sec=0.005):
        """"
        Args:
            robot:       A MultibodyPlant.
            lower_limit: A scalar or vector of length robot.num_positions().
                         The lower limit of the slider will be the maximum
                         value of this number and any limit specified in the
                         Joint.
            upper_limit: A scalar or vector of length robot.num_positions().
                         The upper limit of the slider will be the minimum
                         value of this number and any limit specified in the
                         Joint.
            resolution:  A scalar or vector of length robot.num_positions()
                         that specifies the discretization of the slider.
            length:      The length of the sliders (passed as an argument to
                         tk.Scale).
            update_period_sec: Specifies how often the window update() method
                         gets called.
        """
        VectorSystem.__init__(self, 0, robot.num_positions())

        def _reshape(x, num):
            x = np.array(x)
            assert len(x.shape) <= 1
            return np.array(x) * np.ones(num)

        lower_limit = _reshape(lower_limit, robot.num_positions())
        upper_limit = _reshape(upper_limit, robot.num_positions())
        resolution = _reshape(resolution, robot.num_positions())

        # Schedule window updates in either case (new or existing window):
        self.DeclarePeriodicPublish(update_period_sec, 0.0)

        self._slider = []
        self._slider_position_start = []
        context = robot.CreateDefaultContext()
        state = robot.GetPositionsAndVelocities(context)
        self._default_position = state[:robot.num_positions()]

        k = 0
        for i in range(0, robot.num_joints()):
            joint = robot.get_joint(JointIndex(i))
            low = joint.position_lower_limits()
            upp = joint.position_upper_limits()
            for j in range(0, joint.num_positions()):
                self._slider_position_start.append(joint.position_start() + j)
                self._slider.append(FloatSlider(min=max(low[j],
                                                       lower_limit[k]),
                                max=min(upp[j], upper_limit[k]),
                                step=resolution[k],
                                continuous_update=True,
                                description=joint.name(),
                                style = {'description_width': 'initial'},
                                layout=Layout(width=f"'{length}'")))
                display(self._slider[k])
                k += 1

    def set_position(self, q):
        """
        Set all robot positions (corresponding to joint positions and
        potentially positions not associated with any joint) to the
        values in q.

        Args:
            q: a vector whose length is robot.num_positions().
        """
        self._default_position = q
        for i in range(len(self._slider)):
            self._slider[i].value = q[self._slider_position_start[i]]

    def set_joint_position(self, q):
        """
        Set the slider positions to the values in q.

        Args:
            q: a vector whose length is the same as the number of joint
            positions (also the number of sliders) for the robot.
        """
        assert(len(q) == len(self._default_position))
        for i in range(len(self._slider)):
            self._slider[i].value = q[i]

    def DoPublish(self, context, event):
        process_ipywidget_events()

    def DoCalcVectorOutput(self, context, unused, unused2, output):
        output[:] = self._default_position
        for i in range(0, len(self._slider)):
            output[self._slider_position_start[i]] = self._slider[i].value


# Robot Arms

The next two cells will give you a simple interface to move the joints around on a variety of robot arm models.

Have a favorite robot that I haven't included here?  If you send me a link to a properly licensed URDF or SDF description of the robot and it's mesh files, I'm happy to add it!

Note: The Jaco doesn't visualize properly in this renderer yet.  See drake issue [#13846](https://github.com/RobotLocomotion/drake/issues/13846).

In [ ]:
from IPython.display import display
from ipywidgets import Dropdown, Layout
from pydrake.common import FindResourceOrThrow

arm_selector = Dropdown(options=["Kuka LBR iiwa 7",
                        "Kuka LBR iiwa 14",
                        "Franka Emika Panda",
                        "Kinova Jaco Gen2 (7 DoF)"],
               description="Choose your robot arm",
               style = {'description_width': 'initial'})
display(arm_selector)

def get_model_file(description):
  # Note: I could download remote model resources here if necessary.
  if description == "Kuka LBR iiwa 7":
    return FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
  elif description == "Kuka LBR iiwa 14":
    return FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")
  elif description == "Franka Emika Panda":
    return FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm_hand.urdf")
  elif description == "Kinova Jaco Gen2 (7 DoF)":
    return FindResourceOrThrow("drake/manipulation/models/jaco_description/urdf/j2s7s300.urdf")
  raise Exception("Unknown model")


In [ ]:
from IPython.display import display
from ipywidgets import ToggleButton
import numpy as np
from pydrake.all import (
    DiagramBuilder, SceneGraph, MultibodyPlant, MultibodyPositionToGeometryPose, 
    ConnectMeshcatVisualizer, Parser, Simulator)

builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(time_step=0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant, scene_graph).AddModelFromFile(get_model_file(arm_selector.value))
plant.Finalize()

# Add sliders to set positions of the joints.
sliders = builder.AddSystem(JointSliders(robot=plant))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
builder.Connect(sliders.get_output_port(0), to_pose.get_input_port())
builder.Connect(
    to_pose.get_output_port(),
    scene_graph.get_source_pose_port(plant.get_source_id()))

ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

# Make the diagram and run it.
diagram = builder.Build()
simulator = Simulator(diagram)

simulator.set_target_realtime_rate(1.0)

stop_button = ToggleButton(value=False, description='Stop Simulation')
display(stop_button)
while not stop_button.value:
  simulator.AdvanceTo(simulator.get_context().get_time() + 5.0)
stop_button.value = False

# Simulation with Block Diagrams

In this chapter, and throughout the notes, we will be building up our simulations and controllers using drake's block diagram modeling language (we call it the "systems framework").  [This tutorial](https://mybinder.org/v2/gh/RobotLocomotion/drake/nightly-release?filepath=tutorials/dynamical_systems.ipynb)  provides a brief introduction.  


# MultibodyPlant and SceneGraph

The first thing we need in most simulations for manipulation is our model of the world.  In drake, we will model the world using two systems: [MultibodyPlant](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html) (the physics engine) and [SceneGraph](https://drake.mit.edu/doxygen_cxx/classdrake_1_1geometry_1_1_scene_graph.html) (the geometry engine).  These systems are kept logically separate for a reason; in many applications (like autonomous driving) we want all of the collision detection / rendering features of the geometry engine, but want to supply our own simple models for the dynamics.  But in these notes, we will almost always use them together.  

Let's start by adding just a single object into the world.

In [ ]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, FindResourceOrThrow,
    Parser
)

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModelFromFile(
        FindResourceOrThrow(
            "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf"))
plant.Finalize()


# 3D visualization

In order to view the world, we need to add a visualizer.  We support a number of visualizers in drake (and even ship a simple VTK-based "drake-visualizer" with our binary releases).  But because we're working in a jupyter notebook, and want you to be able to run 3d graphics on any machine without any installation required, we'll use the [MeshCat](https://github.com/rdeits/meshcat) visualizer throughout these notes.  

Let's create the visualizer and add it to our diagram.

In [ ]:
from pydrake.all import ConnectMeshcatVisualizer

meshcat = ConnectMeshcatVisualizer(builder, scene_graph)

If you click on the link above, you can open a new browser window to see the visualization.

But the scene will be empty so far.  We need to actually construct the diagram and ask it to publish.  (MeshcatVisualizer will automatically publish on a fixed period during simulation).

In [ ]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.Publish(context)

 # Simulating RGB-D cameras

The 3D visualization is mean for the user.  But we want a similar tool that can model what a camera at a known position in the environment would see.  

As we will see later, we can also render the output of these cameras as a point cloud.

# Adding a robot (and its low-level controller)


# Running the simulation.

We've put together quite a complex system now.  Let's make sure we understand the diagram that we've built by visualizing the resulting block diagram.

In [ ]:
from IPython.display import HTML

#HTML(GenerateHtml(diagram))